In [3]:
#Set environment options
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

In [2]:
#function to combine matchup data
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        #pass
        return None
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [5]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import time

#get nba team data including id's
nba_teams = pd.DataFrame(teams.get_teams())
#Create list of NBA team id's
team_ids = nba_teams['id'].tolist()

#season data should be pulled for 
s = '2022-23'


#function to call api and get all NBA games for a given season
def get_all_games(team__ids,season):
    team_season = []
    for i in team_ids:
        games = leaguegamefinder.LeagueGameFinder(team_id_nullable = i,season_nullable = season,season_type_nullable ='Regular Season').get_data_frames()[0]
        team_season.append(games)
        nba_season = pd.concat(team_season)
        time.sleep(1)
    return(nba_season)

nba_season = get_all_games(team_ids,s)
nba_season = nba_season.sort_values(by=['GAME_DATE','GAME_ID'])

In [5]:
nba_season = nba_season.sort_values(by=['GAME_DATE','GAME_ID'])
nba_season.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
14,22022,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240,126,46,82,0.561,12,35,0.343,22,28,0.786,6,30,36,24,8,3,10,24,9.0
13,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0
12,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,241,109,40,94,0.426,10,40,0.250,19,25,0.760,9,39,48,23,12,4,21,18,-14.0
14,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,45,99,0.455,16,45,0.356,17,23,0.739,11,37,48,31,11,4,18,23,14.0
14,22022,1610612765,DET,Detroit Pistons,0022200003,2022-10-19,DET vs. ORL,W,239,113,40,94,0.426,14,38,0.368,19,24,0.792,12,29,41,31,11,4,12,21,4.0


In [15]:
home_team = nba_season[nba_season.MATCHUP.str.contains(' vs. ')]
away_team = nba_season[nba_season.MATCHUP.str.contains(' @ ')]

In [28]:
home_team_1 = home_team[['SEASON_ID','GAME_ID','GAME_DATE','TEAM_ID','TEAM_ABBREVIATION','MATCHUP','WL']]
home_team_1= pd.get_dummies(home_team_1, columns=['TEAM_ABBREVIATION','WL'], prefix=['HOME','HOME'])
home_team_1 = home_team_1.drop(['HOME_L'],axis=1)
home_team_1 = home_team_1.rename(columns={'TEAM_ID':'HOME_TEAM_ID','HOME_W':'HomeWIN = 1'})


away_team_1 = away_team[['GAME_ID','TEAM_ID','TEAM_ABBREVIATION']]
away_team_1 =  pd.get_dummies(away_team_1, columns=['TEAM_ABBREVIATION'], prefix=["AWAY"])

In [35]:
from nba_api.stats.endpoints import teamdashboardbyopponent

call = teamdashboardbyopponent.TeamDashboardByOpponent(team_id=1610612756,date_to_nullable='2022-10-19')
overall_team_dashboard = call.get_data_frames()[0]
#conference_team_dashboard = call.get_data_frames()[1]
#division_team_dashboard = call.get_data_frames()[2]
#opponent_team_dashboard = call.get_data_frames()[3]

overall_team_dashboard.head()
#conference_team_dashboard.head()
#division_team_dashboard.head()
#opponent_team_dashboard.head()

,GROUP_SET,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,Overall,2022-23,1,1,0,1.0,48.0,40,85,0.471,8,22,0.364,19,22,0.864,8,32,40,25,12.0,4,5,4,29,22,107,2.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,167,2022-23


In [37]:
gameDate_homeTeam = home_team_1[['GAME_DATE','HOME_TEAM_ID']]
#gameDate_homeTeam.head()


def team_stats_pergame(gameDate_homeTeam):
    teamGame_stats = []

    for gdate,id in gameDate_homeTeam:
        call = teamdashboardbyopponent.TeamDashboardByOpponent(team_id=id,date_to_nullable= gdate)
        overall_team_dashboard = call.get_data_frames()[0]
        teamGame_stats = teamGame_stats.append(overall_team_dashboard)
        teamGame_stats = pd.concat(teamGame_stats)
        time.sleep(1)
    return(teamGame_stats)

gameDashboard = team_stats_pergame(gameDate_homeTeam)
gameDashboard.head()

ValueError: too many values to unpack (expected 2)

In [55]:
for row in gameDate_homeTeam.iterrows():
    
    print(row[1])

GAME_DATE       2022-10-18
HOME_TEAM_ID    1610612738
Name: 14, dtype: object
GAME_DATE       2022-10-18
HOME_TEAM_ID    1610612744
Name: 14, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612765
Name: 14, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612754
Name: 12, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612737
Name: 14, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612751
Name: 15, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612748
Name: 14, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612761
Name: 15, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612763
Name: 14, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612750
Name: 14, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612759
Name: 15, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612762
Name: 15, dtype: object
GAME_DATE       2022-10-19
HOME_TEAM_ID    1610612756
Name: 13, 